# Project EShop

This is a project in which we'll put our newly learned database skills to use. The set up fairly simple: Start the project, make a products app, run migrations and done!

In [ ]:
django-admin startproject EShop
cd EShop
python manage.py startapp products
python manage.py migrate

Then we move on to working with Template layer, make some `HTML` files in the templates directory in products app.

Along with some CSS files which are not the main target of this lesson.

Make the views and set up the urls and finally run the server and the set up is complete.

In [ ]:
# products -> views.py

from django.shortcuts import render

def products(request):
    return render(request, 'products/products.html')

def product_details(request, id):
    return render(request, 'products/product_details.html')

In [ ]:
python manage.py runserver

### Designing the Database

The main target of this subject, as spoken before, is to get the details of some products from database and show them in HTML webpage (and vice versa). Meaning we will basically need to build the flow of data among all three layers of Django: Model, Template, View!

We'll start by making a model for our products just like the previous subject; migrate them to the database and then add some sample products to the database using Django Shell.

In [ ]:
# prodcuts -> models.py

from django.db import models

class Product(models.Model):
    name = models.CharField(default="N/A", max_length=100)
    price = models.IntegerField(default=0)
    is_active = models.BooleanField(default=True)

In [ ]:
python manage.py makemigrations
pyhton manage.py migrate
python manage.py shell

### Product List Page

Now we have completed our work with the Model layer. Time for View and Template. Suppose the body of `products.html` looks something like this:

In [ ]:
{% block body_refs %}

<h1>Hello, World!</h1>

<ul>
    <li><a href="">Item 1</a></li>
    <li><a href="">Item 2</a></li>
    <li><a href="">Item 3</a></li>
</ul>

{% endblock %}

The goal is to have thos `<li>` tags fetch their data from the database -> Product Model. Let's go back to views file and fetch the data there.

In [ ]:
def products(request):
    products = Product.objects.all()
    return render(
        request, 
        'products/products.html', 
        context={'products':products}
    )

You can see that everything we're using here is taught before, it's just the matter of mixing our knowledge into something practical.

Next up, alter the `products.html` file with some Python for loop to do the thing.

In [ ]:
{% block body_refs %}
<h1>Hello, World!</h1>

<ul>
    {% for prod in products %}
        <li>
            <a href="">
                {{ prod.name }}
            </a>
        </li>
    {% endfor %}
</ul>
{% endblock %}

Run the server once again to see the magic!

### Product Details Page

This next step is faily simple, we just need to build the connection of each `<a>` tag in product list to the relevant page of the desired product.

The key in doing so is the Primary Key in the database, which was id and was generated automatically after migration.

We'll start by adding the URL to the tag itself:

In [ ]:
<ul>
    {% for prod in products %}
        <li>
            <a href="{% url 'product_details' id=prod.id %}">
                {{ prod.name }}
            </a>
        </li>
    {% endfor %}
</ul>

Then we move onto completing the `product_details()` view. If you have completed previous subjects cautiosly, you shouldn't have any problem understanding what's going on.

In [ ]:
def product_details(request, id):
    product = Product.objects.get(id=id)
    return render(
        request, 
        'products/product_details.html', 
        {'product':product}
    )

Finally, since styling isn't the target of this topic, we'll finish the work by a simple outline for `product_details.html` file.

In [ ]:
{% extends 'base.html' %}
{% load static %}

{% block head_refs %}
<link rel="stylesheet" href="{% static '/products/css/products.css' %}">
{% endblock %}

{% block title %}
{{ product.name }}
{% endblock %}

{% block body_refs %}
<h1>{{ product.name }}</h1>
<h2>{{ product.price }}</h2>
{% endblock %}

Restart the app if needed, and see what happens...

If you're a best-practice nerd like me and want to strictly follow the standards all the time, you gotta handle situations where the product does not exist in the database.

In [ ]:
from django.http import Http404
from django.shortcuts import get_object_or_404

def product_details(request, id):
    try:
        product = Product.objects.get(id=id)
    except:
        raise Http404
    
    # or simply
    product = get_object_or_404(Product, pk=id) # WHERE Primary Key = id
    
    return render(
        request, 
        'products/product_details.html', 
        {'product':product}
    )